# zonally periodic jet with internal waves configuration

Description of the configuration, see:

- [Ponte and Klein 2017](https://agupubs.onlinelibrary.wiley.com/doi/full/10.1002/2016JC012214)

- [Ponte and Klein 2013](https://link.springer.com/article/10.1007/s10236-013-0611-7)

The cpp key is JETN
The type of jet (1 to 4) is defined in `cppdefs.h`
Run time options are found in `croco.in`

---
### Internal wave forcing
The forcing is a zonally uniform force along the y direction, which avoids the generation of potential vorticity:

$$
\begin{align}
\mathcal{F}_y(x,y,z,t) &amp= \sum_n f_n(x,y,t) \phi_n(z), \\
f_n(x,y,t) &amp= \frac{1}{h}\int \phi_n(z) \mathcal{F}_y(x,y,z,t) dz, \\
\end{align}
$$
**forcing_itide=2** (in croco.in): We'll assume the forcing is a truncated projection of a dirac delta function:

$$
\begin{align}
\mathcal{F}_y^N(x,y,z,t) &amp= f(x,y,t) \sum_{n\le N} f_n \phi_n(z), \\
f_n &amp=  \frac{1}{h}\int \phi_n(z) \delta(z)  dz, \\
&amp= \frac{1}{h} \phi_n(z=0)
\end{align}
$$
We may need to renormalize in order to modulate the global internal wave energy.

The horizontal and temporal structure of the forcings is given by:

$$
\begin{align}
f(x,y,t) &amp=  f_0 e^{-(y-y_0)^2/L_y^2} \cos (\omega t)
\end{align}
$$

---
### vertical modes
$\phi$ is the vertical structure function of the (u,v,p) and satisfies:

$$
\begin{align}
\frac{d}{dz} \Big (  \frac{1}{N^2} \frac{d}{dz} \phi  \Big )
+ \frac{1}{c^2} \phi = 0, \\
g \frac{d}{dz} \phi + N^2 \phi = 0 
\;\; \mathrm{at} \;\; z=0 \\
\frac{d}{dz} \phi = 0 
\;\; \mathrm{at} \;\; z=-h
\end{align}
$$
where c is the eigenvalue to be determined. Variables will be projected according to:

$$
\begin{align}
\big [ u, v, p \big ] = 
\sum_m \big [ \hat{u}_m , \hat{v}_m , \hat{p}_m \big ]
\times \phi_m \\
\big [ \rho, w \big ] = 
\sum_m \big [ \hat{\rho}_m, \frac{1}{N^2(z)}\hat{w}_m  \big ]
\times  \frac{d}{dz}\phi_m
\end{align}
$$
Normalization is such that:

$$
\begin{align}
\frac{1}{h}\int \phi_m^2 dz &amp;= 1, \\
\frac{1}{h}\int \frac{1}{N^2} \Big ( \frac{d}{dz} \phi_n \Big )^2 dz &amp;= \frac{1}{c_n^2} - \frac{\phi_n^2(0)}{gh}.
\end{align}
$$
References: Gill p160, Kloosterziel 1995 ($\hat{p}$ of Gill is $\phi$ of kloorsterziel)

---
### relaxation

3 levels for relaxation:
- global relaxation
- zonal mean damping
- zonal anomalies damping

---

### compilation

In `configs/jetn/`, run:

```
./jobcomp
```

If few files have been modified, just run:
```
./jobmake
```

In [25]:
### default croco.in

#nudg_cof:    TauT_in, TauT_out, TauM_in, TauM_out  [days for all]
#              1.      100.0e+0       1.    100.0e+0

#itide:  flag_itide   omega_itide   nmode_itide  forcing_itide  Az_itide  y_itide    start_time_itide
#            T      1.45444104333e-4     2         1            0.0e-2   400.0e+3       4000

#itidep: flag_ptb   dcos_ptb   dsin_ptb   nmode_ptb    scale_ptb
#            0       -1.0e-6     0.0e-6       2           1.0e0

#jet: flag_jet   flag_jet_perturb  jet_ywidth  jet_weight jet_decay_start
#       T              T            800.0e+3      0.5        500

#damp_rl: rl_flag_south rl_L_south rl_tau_in_south rl_tau_out_south rl_flag_north rl_L_north rl_tau_in_north rl_tau_out_north
#              T        1000.0e+3     350.0e+3      1000.0e+0    1.0e+0           T             1000.0e+3     350.0e+3   1000.0e+0       1.0e+0

#wind_forcing: tau_max (m/s) T_pulse (hours) T_cylce(days)
#	          0.0	        12              10 


In [30]:
# Jet
J = {0: {'jet': ['F', 'F', 800.0e+3, 0.5, 500]},            # sans jet
    }

# Amplitude and start of Tide (Az_itide, start_time_itide, y_itide)
T = {0: {'itide': ['T', 1.45444104333e-4, 2, 1, 2.0e-3, 1500.0e+3, 0]},
     1: {'itide': ['T', 1.45444104333e-4, 1, 2, 2.0e-3, 1500.0e+3, 0]},
     2: {'itide': ['T', 1.45444104333e-4, 2, 2, 2.0e-3, 1500.0e+3, 0]},
     3: {'itide': ['T', 1.45444104333e-4, 4, 2, 2.0e-3, 1500.0e+3, 0]},
     4: {'itide': ['T', 1.45444104333e-4, 8, 2, 2.0e-3, 1500.0e+3, 0]},
    }

# Wind forcing
W = {0: {'wind_forcing': [0.1, 12, 10]},      
    }

# Global Relaxation
R = {0: {'nudg_cof':    [0., 0.0e+0, 0., 0.0e+0]},      # no global relaxation
    }

---

### launch: python code

In [31]:
import crocosi.launch as lc
import numpy as np
import itertools

workdir = '/home1/scratch/slgentil'

# parameters
wp=5

# restart
rst = workdir+'jet_cfg1_wp{}_4km_0a1000j/t5/jetn_rst*.nc'.format(wp)

Ts = {k:v for k,v in T.items() if k in [1,2,3,4]}
#for m,v in M.items():
for j, t, r in itertools.product(J,Ts,R):
#for j, t in itertools.product(J,T):
    lc.run('jet_cfg1_wp{}_tide{}_jet{}_relax{}_zmean+lin_4km_0a20j'.format(wp,t,j,r),
           jobname='f{}{}{}'.format(j,t,r),
           workdir=workdir,
           nbchains=1,
           restart = False,
           #restart = rst,
           launch=True,
           **J[j], **Ts[t], **R[r])

Run will be stored in /home1/scratch/slgentil/jet_cfg1_wp5_tide1_jet0_relax0_zmean+lin_4km_0a20j
Run will be stored in /home1/scratch/slgentil/jet_cfg1_wp5_tide2_jet0_relax0_zmean+lin_4km_0a20j
Run will be stored in /home1/scratch/slgentil/jet_cfg1_wp5_tide3_jet0_relax0_zmean+lin_4km_0a20j
Run will be stored in /home1/scratch/slgentil/jet_cfg1_wp5_tide4_jet0_relax0_zmean+lin_4km_0a20j


In [28]:
forcing_tide=1

Ts = {k:v for k,v in T.items() if k in [forcing_tide]}
#for m,v in M.items():
for j, t, r in itertools.product(J,Ts,R):
    print(j,t,r)

0 1 0


### launch: manual commands

```
./jobcomp
python chain_datarmor.py jet_cfg1_wp75_4km_0a1000j 10 03:00:00 4 jetn 0
# and release
```

Restart:
```
python chain_datarmor.py jet_cfg1_wp75_4km_m1_500a1000j 5 03:00:00 4 jetn 1
# follow instructions and release
```

---

## list of simulations available on datarmor:

**Please send email to aurelien to update this note if you see errors !**

In `/home/c11-data/Test_aponte/jetn`:

| Name  | Resolution (dx=dy) | Length (days) | Jet configuration | Weight | Internal tide | Mode number| Amplitude |  High resolution output| Other |
|--|:-:|:-:|:-:|:-:|:-:|:-:|:-:|:-:|:-:|
| jet_cfg1_wp4_4km_0a1000j | 4 km | 1000d |  1 | 0.4  | No  |  | | No | |
| jet_cfg1_wp4_4km_500a1000j | 4 km | 500d |  1 | 0.4  | No  |  | | Yes | floats |
| jet_cfg1_wp4_m1a0_4km_500a1000j | 4 km | 500d |  1 | 0.4  | Yes  | 1 | 0.01 | Yes | floats |
| jet_cfg1_wp5_4km_0a1000j | 4 km | 1000d |  1 | 0.5  | No  |  | | No | |
| jet_cfg1_wp5_4km_500a1000j | 4 km | 500d |  1 | 0.5  | No  |  | | Yes | floats |
| jet_cfg1_wp5_4km_m1a1_500a1000j | 4 km | 500d |  1 | 0.5  | Yes  | 1 | 0.02 | Yes | floats |
| jet_cfg1_wp5_4km_m2a1_500a1000j | 4 km | 500d |  1 | 0.5  | Yes  | 2 | 0.02 | Yes | floats |
| jet_cfg1_wp6_4km_0a1000j | 4 km | 1000d |  1 | 0.6  | No  |  | | No | |
| jet_cfg1_wp6_4km_500a1000j | 4 km | 500d |  1 | 0.6  | No  |  | | Yes | floats |
| jet_cfg1_wp6_4km_m1a0_500a1000j | 4 km | 500d |  1 | 0.6  | Yes  | 1 | 0.01 | Yes | floats |
| jet_cfg1_wp6_2km_m1a0_500a700j | 2 km | 200d |  1 | 0.6  | Yes  | 1 | 0.01 | Yes | floats |
| jet_cfg1_wp6_4km_m1a1_500a1000j | 4 km | 500d |  1 | 0.6  | Yes  | 1 | 0.02 | Yes | floats |
| jet_cfg1_wp6_4km_m2a1_500a1000j | 4 km | 500d |  1 | 0.6  | Yes  | 2 | 0.02 | Yes | floats |
| jet_cfg1_wp7_4km_0a1000j | 4 km | 1000d |  1 | 0.7  | No  |  | | No | |
| jet_cfg1_wp8_4km_0a1000j | 4 km | 1000d |  1 | 0.8  | No  |  | | No | |
| jet_cfg1_wp8_4km_500a1000j | 4 km | 500d |  1 | 0.8  | No  |  | | Yes | floats |
| jet_cfg1_wp8_4km_m1a1_500a1000j | 4 km | 500d |  1 | 0.8  | Yes  | 1 | 0.02 | Yes | floats |
| jet_cfg1_wp8_4km_m2a1_500a1000j | 4 km | 500d |  1 | 0.8  | Yes  | 2 | 0.02 | Yes | floats |
| jet_cfg1_wp9_4km_0a1000j | 4 km | 1000d |  1 | 0.9  | No  |  | | No | |

Simulations with an **unperturbed jet** in `/home/c11-data/Test_aponte/jetn`:

| Name  | Resolution (dx=dy) | Length (days) | Jet configuration | Weight | Internal tide | Mode number| Amplitude |  High resolution output| Other |
|--|:-:|:-:|:-:|:-:|:-:|:-:|:-:|:-:|:-:|
| jet_cfg1_wp0_m1a1_4km_0a100j | 4 km | 100d |  1 | 0.0  | Yes  | 1 | 0.02 | Yes | floats |
| jet_cfg1_wp0_m1a1_2km_0a100j | 2 km | 100d |  1 | 0.0  | Yes  | 1 | 0.02 | Yes | floats |
| jet_cfg1_wp75_unperturbed_m1a1_4km_0a100j | 4 km | 100d |  1 | 0.75  | Yes  | 1 | 0.02 | Yes | floats, jet unperturbed initially |
| jet_cfg1_wp75_unperturbed_m1a1_2km_0a100j | 2 km | 100d |  1 | 0.75  | Yes  | 1 | 0.02 | Yes | floats, jet unperturbed initially |


**Notes:**
- The weight modulates the strength of the turbulence
- internal tide modes are: 0=barotropic, 1=1st baroclinic, 2=2nd baroclinic, etc
- internal tide amplitude is has given in `croco.in` (Az_itide parameter) and may modified online (for different modes)

---

## outputs:

see [`iodef.xml`](iodef.xml)

### file_his_*.nc

Relevant temporal variable: `time_counter`

Variables are 2D or 3D:

- `v_a` : temporal averaged of variable $v$

- `v_t_cos` : temporal averaged of $v\times\cos(\omega t)$

- `v_t_sin` : temporal averaged of $v\times\cos(\omega t)$

- `v_t_dirac` : instantaneous value of $v$ at the center of the interval

The relevant time interval for this output has size `freq_op` (2d typically) and is outputed every `output_freq` (25d typically).

![his](croco_jetn_time.001.png)


### file_ave_*.nc

Relevant temporal variable: `time_counter`

Variables are 2D and averaged over a temporal window of size `output_freq` (2d typically).

- `v` : temporal averaged of variable $v$

- `v_t_cos` : temporal averaged of $v\times\cos(\omega t)$

- `v_t_sin` : temporal averaged of $v\times\cos(\omega t)$

![ave](croco_jetn_time.002.png)

### file_surf_*.nc

Relevant temporal variable: `time_instant`

Variables are 2D (surface) and instantaneous every `output_freq` (10 min typically).

### file_inst_*.nc, file_sta1_*.nc, ...

Relevant temporal variable: `time_instant`

Variables are 1D and instantaneous every `output_freq` (30min typically).

### file_swath_*.nc

Not that useful for now.


### nicer treatment of time coordinate

Work is ongoing around xarray in order to implement useful features for this
such as selection alon non-dim coordinates [issue](https://github.com/pydata/xarray/issues/1603)